In [1]:
# %%
# Env
import dotenv
dotenv.load_dotenv()

import sys, os
dir_head = os.environ.get("REPO_DIR", ".")
sys.path.insert(0, os.path.join(dir_head, "src"))

import numpy as np
from pixell import enmap

import warnings
import itertools
from scipy.ndimage import gaussian_filter
import multiprocessing
from functools import partial

warnings.filterwarnings("ignore")

# Files from the src dir in git
import utils as ut
import covariance as cov

import matplotlib.pyplot as plt
import random


In [2]:
# Setup
config_data_fname = os.environ["REPO_DIR"] + "/configs/config_mcmc_ajay.yaml"
config_data = ut.get_config_file(config_data_fname)

cluster_region = ut.get_region(region_center_ra=config_data['region_center_ra'], 
                               region_center_dec=config_data['region_center_dec'],
                               region_width=config_data['region_width'])

fname = os.getenv("ACT_DATADIR") + "act_cut_dr6v2_pa5_f150_4way_coadd_map_srcfree.fits"

save_cov = 1
plot_1d = 0
plot_2d = 0

data_ref = ut.imap_dim_check(enmap.read_map(fname=fname, box=cluster_region))
data_shape, data_wcs = data_ref.shape, data_ref.wcs

# if save_cov: 
#     os.system(f"rm -rf {os.environ['COV_DIR']}/cov_*npy*")

In [ ]:
# Calculate covariances
combos = list(itertools.product(config_data['data'], config_data['data']))

for combo in combos:
    freq1 = combo[0].split('_')[0]
    array1 = combo[0].split('_')[1]
    inst1 = combo[0].split('_')[2]
    scan1 = combo[0].split('_')[3]

    freq2 = combo[1].split('_')[0]
    array2 = combo[1].split('_')[1]
    inst2 = combo[1].split('_')[2]
    scan2 = combo[1].split('_')[3]

    if inst1 == 'act':
        data_dir1 = os.getenv("ACT_DATADIR")
    elif inst1 == 'planck':
        data_dir1 = os.getenv("PLANCK_DATADIR")

    if inst2 == 'act':
        data_dir2 = os.getenv("ACT_DATADIR")
    elif inst2 == 'planck':
        data_dir2 = os.getenv("PLANCK_DATADIR")

    print("Calculating covariance for: {}".format(combo))

    mean_tpsd, mean_npsd, mean_spsd, all_regions_npsd, all_regions_spsd = cov.get_covariance(freq1=freq1, 
                                                                                freq2=freq2,
                                            
                                                                                array1=array1, 
                                                                                array2=array2,
                                            
                                                                                inst1=inst1, 
                                                                                inst2=inst2,
                                            
                                                                                data_dir1=data_dir1,
                                                                                data_dir2=data_dir2,
                                                                                
                                                                                data_wcs=data_wcs,
                                                                                
                                                                                cf=config_data)
    
    if plot_1d:
        #b_npsd, l_npsd = enmap.lbin(map=np.abs(mean_npsd), bsize=binsize)
        b_tpsd, l_tpsd = enmap.lbin(map=np.abs(mean_tpsd), bsize=binsize)
        #b_spsd, l_spsd = enmap.lbin(map=np.abs(mean_spsd), bsize=binsize)

        plt.figure(figsize=(7, 3))
        #plt.loglog(l_npsd, b_npsd, label="Noise")
        plt.loglog(l_tpsd, b_tpsd, label="Total")
        #plt.loglog(l_spsd, b_spsd, label="Signal")
        plt.title(f"Combo: {combo}: Absolute value of the power spectrum", fontsize=10)
        plt.legend(fontsize=8)
        plt.grid()
        plt.show()

    

Calculating covariance for: ('150_pa4_act_dr6v2', '150_pa4_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('150_pa4_act_dr6v2', '220_pa4_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('150_pa4_act_dr6v2', '150_pa5_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('150_pa4_act_dr6v2', '98_pa5_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('220_pa4_act_dr6v2', '150_pa4_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('220_pa4_act_dr6v2', '220_pa4_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('220_pa4_act_dr6v2', '150_pa5_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('220_pa4_act_dr6v2', '98_pa5_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('150_pa5_act_dr6v2', '150_pa4_act_dr6v2')
Smoothing the total power spectrum.
Calculating covariance for: ('150_pa5_a